# Grounding Schematic Representations

Some modeling languages such as PDDL allow the use of first-order variables in order to compactly represent
several actions and state variables that have a similar form. Most planners, however, work on the ground representation,
where all variables have been appropriately substituted by type-consistent objects.

Tarski allows the grounding of such lifted representations by implementing a variation of the strategy
implemented by Malte Helmert in the Fast Downward planner
(Helmert, Malte.
"[Concise finite-domain representations for PDDL planning tasks](https://ai.dmi.unibas.ch/papers/helmert-aij2009.pdf)."
_Artificial Intelligence_ 173.5-6 (2009): 503-535).
Fast Downward incrementally grounds only those atoms and ground actions that are determined to be
_reachable from the initial state of the problem_. This removes from consideration a (sometimes) large
number of ground elements that will never be relevant for the problem. Determining such reachability precisely
is as hard as planning itself, but Fast Downward overapproximates the set of reachable elements using the delete-free
relaxation of the problem.

Helmert's algorithm is based on a logic program (LP) that encodes such notion of reachability, and whose canonical
model precisely encodes the set of reachable ground atoms and actions of the problem.
Whereas Fast Downward solves the LP itself, Tarski generates the program and then relies on an off-the-shelf
answer set solver to find the model for it. In particular, Tarski uses the tools provided by the 
[Potassco ASP suite](https://potassco.org/). These are neatly packaged for e.g. different versions of Ubuntu and can be 
installed easily with `sudo apt install gringo`.

Let us quickly show how to ground a standard Gripper problem encoded with the standard schematic representation:

In [ ]:
from tarski.io import PDDLReader

reader = PDDLReader(raise_on_error=True)
reader.parse_domain('./benchmarks/gripper.pddl')
problem = reader.parse_instance('./benchmarks/gripper_prob01.pddl')
lang = problem.language


We can get hold of the classes encapsulating the grounding process from the `tarski.grounding` module

In [ ]:
from tarski.grounding import LPGroundingStrategy
from tarski.grounding.errors import ReachabilityLPUnsolvable

and ground the instance of Blocks World with a one-liner

In [ ]:
grounder = LPGroundingStrategy(reader.problem)


We can inspect the results of the grounding process with ease. For the ground actions, 
we can query the `grounding` object for a dictionary that maps every action schema to
a list tuples of object names that schema variables are to be bound to, as they were 
found to be reachable

In [ ]:
try:
    actions = grounder.ground_actions()   
    for name, bindings in actions.items():
        print(f'Action schema {name} has {len(bindings)} reachable bindings:')
        print(", ".join(map(str, bindings)))
except ReachabilityLPUnsolvable:
    print("Problem was determined to be unsolvable during grounding")


Note that the grounder is sometimes able to determine that an instance is unsolvable during its reachability analysis,
in which case it throws an exception.
The astute reader may be now wondering why we only return the _bindings_ rather
than the grounded actions themselves. The reason is that doing so is not safe
in general, as big instances (such as those commonly found on the IPC-18 benchmarks)
result in thousands of ground operators, and is possible to exhaust the memory
available to the Python interpreter. 

To ameliorate that issue, we settle for returning instead the minimal amount of data
necessary so users can decide how to instantiate ground operators efficiently.  

To access the ground atoms, or _state variables_, identified during the grounding
process, we use a similar interface

In [ ]:
try:
    lpvariables = grounder.ground_state_variables()    
    for i, atom in lpvariables.enumerate():
        print(f'Atom #{i}: {atom}')
except ReachabilityLPUnsolvable:
    print("Problem was determined to be unsolvable during grounding")

We note that in this case we _do_ return the actual grounded language element. This
is because the number of fluents is not generally subject to the same kind of 
combinatorial explosion that ground actions are. **This assessment may change
in the future**.